In [1]:
!pip install sdv

^C


   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
    --------------------------------------- 0.3/14.6 MB ? eta -:--:--
   -- ------------------------------------- 1.0/14.6 MB 2.8 MB/s eta 0:00:05
   ----- ---------------------------------- 1.8/14.6 MB 3.6 MB/s eta 0:00:04
   ------- -------------------------------- 2.6/14.6 MB 3.5 MB/s eta 0:00:04
   --------- ------------------------------ 3.4/14.6 MB 3.6 MB/s eta 0:00:04
   ------------ --------------------------- 4.5/14.6 MB 3.9 MB/s eta 0:00:03
   --------------- ------------------------ 5.5/14.6 MB 4.0 MB/s eta 0:00:03
   ------------------ --------------------- 6.8/14.6 MB 4.3 MB/s eta 0:00:02
   --------------------- ------------------ 7.9/14.6 MB 4.3 MB/s eta 0:00:02
   ------------------------ --------------- 8.9/14.6 MB 4.5 MB/s eta 0:00:02
   ---------------------------- ----------- 10.2/14.6 MB 4.6 MB/s eta 0:00:01
   ------------------------------ --------- 11.0/14.6 MB 4.6 MB/s eta 0:00:01
   --------

In [ ]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import random

#Fetch Data
url = "https://raw.githubusercontent.com/shrikant-temburwar/Loan-Prediction-Dataset/master/train.csv"
df = pd.read_csv(url)

df = df[['Gender', 'Education', 'Self_Employed', 'ApplicantIncome',
         'CoapplicantIncome', 'LoanAmount', 'Credit_History',
         'Property_Area', 'Loan_Status']]

#Clean
df = df.dropna()
df['Credit_History'] = df['Credit_History'].astype(str)

#Train CTGAN
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)
model = CTGANSynthesizer(metadata)
model.fit(df)

#Generate 1000 Synthetic Records
synthetic_data = model.sample(1000)

reasons = [
    "I need to renovate my house immediately.",
    "Planning to expand my small business operations.",
    "Urgent medical expenses covered by loan.",
    "Buying a new car for daily commute.",
    "Consolidating high-interest debts.",
    "Investment in a new startup opportunity."
]

qualifiers = [
    "I am very confident in repayment.",
    "I might struggle slightly initially.",
    "Business is booming right now.",
    "Facing some financial stress currently."
]

def generate_text(row):
    reason = random.choice(reasons)
    if row['ApplicantIncome'] > 10000:
        qualifier = "I am very confident in repayment."
    else:
        qualifier = random.choice(qualifiers)
    return f"{reason} {qualifier}"

synthetic_data['Application_Note'] = synthetic_data.apply(generate_text, axis=1)

#Save
print(synthetic_data.head())
synthetic_data.to_csv('synthetic_financial_data_enhanced.csv', index=False)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


   Gender     Education Self_Employed  ApplicantIncome  CoapplicantIncome  \
0    Male      Graduate            No             1969           0.000000   
1    Male  Not Graduate            No             1544           0.000000   
2    Male      Graduate            No              388         345.682079   
3    Male  Not Graduate           Yes              150         145.880326   
4  Female      Graduate           Yes             1691          32.047091   

   LoanAmount Credit_History Property_Area Loan_Status  \
0       247.0            0.0         Rural           N   
1       268.0            1.0     Semiurban           Y   
2       121.0            1.0     Semiurban           Y   
3       216.0            1.0     Semiurban           Y   
4       103.0            1.0         Rural           N   

                                    Application_Note  
0  Urgent medical expenses covered by loan. Facin...  
1  Urgent medical expenses covered by loan. Busin...  
2  Buying a new car for

In [20]:
# Load the dataset
import pandas as pd
import numpy as np

# Load the CSV file
data = pd.read_csv('/kaggle/input/wie3007-dataset/synthetic_financial_data_enhanced.csv')

# Check for missing values
missing_summary = data.isnull().sum()
print("Missing Values Summary:")
print(missing_summary)


Missing Values Summary:
Gender               0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Credit_History       0
Property_Area        0
Loan_Status          0
Application_Note     0
dtype: int64


In [21]:
categorical_cols = [
    'Gender', 'Education', 'Self_Employed',
    'Property_Area', 'Loan_Status'
]

df[categorical_cols] = df[categorical_cols].astype(object)

In [22]:
# Total household income
df['TotalIncome'] = df['ApplicantIncome'] + df['CoapplicantIncome']

# Loan to income ratio (add 1 to avoid division by zero)
df['LoanIncomeRatio'] = df['LoanAmount'] / (df['TotalIncome'] + 1)

In [24]:
df['HasCreditHistory'] = df['Credit_History'].apply(
    lambda x: 1 if x == 1 else 0
)

In [25]:
df['Employment_Type'] = df['Self_Employed'].map({
    'Yes': 'Self_Employed',
    'No': 'Salaried'
})

In [26]:
df['IncomeLevel'] = pd.cut(
    df['TotalIncome'],
    bins=[0, 2000, 5000, 10000, df['TotalIncome'].max()],
    labels=['Low', 'Medium', 'High', 'Very High']
)

In [27]:
df['Note_Length'] = df['Application_Note'].str.len()

# Risk keyword flag
risk_keywords = ['urgent', 'debt', 'medical', 'delay', 'struggle']

df['Risk_Keyword_Flag'] = df['Application_Note'].str.lower().apply(
    lambda x: 1 if any(word in x for word in risk_keywords) else 0
)

In [28]:
encode_cols = [
    'Gender', 'Education', 'Employment_Type',
    'Property_Area', 'IncomeLevel'
]

df_model = pd.get_dummies(df, columns=encode_cols, drop_first=True)

print("Final feature set shape:", df_model.shape)
df_model.head()

Final feature set shape: (1000, 20)


,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Loan_Status,Application_Note,TotalIncome,LoanIncomeRatio,HasCreditHistory,Note_Length,Risk_Keyword_Flag,Gender_Male,Education_Not Graduate,Employment_Type_Self_Employed,Property_Area_Semiurban,Property_Area_Urban,IncomeLevel_Medium,IncomeLevel_High,IncomeLevel_Very High
0,No,1969,0.000000,247.0,0.0,N,Urgent medical expenses covered by loan. Facin...,1969.000000,0.125381,0,80,1,True,False,False,False,False,False,False,False
1,No,1544,0.000000,268.0,1.0,Y,Urgent medical expenses covered by loan. Busin...,1544.000000,0.173463,1,71,1,True,True,False,True,False,False,False,False
2,No,388,345.682079,121.0,1.0,Y,Buying a new car for daily commute. I might st...,733.682079,0.164697,1,72,1,True,False,False,True,False,False,False,False
3,Yes,150,145.880326,216.0,1.0,Y,I need to renovate my house immediately. Facin...,295.880326,0.727566,1,80,0,True,True,True,True,False,False,False,False
4,Yes,1691,32.047091,103.0,1.0,N,Planning to expand my small business operation...,1723.047091,0.059743,1,88,0,False,False,True,False,False,False,False,False


In [29]:
df_model.to_csv("synthetic_financial_data_feature_engineered.csv", index=False)
print("Feature-engineered dataset saved successfully.")

Feature-engineered dataset saved successfully.
